# Test call Terra api with service account

The following tasks have already been completed. To replicate this analysis with a different service account or project you may need to perform some / all of these tasks. 

- create service account
 - https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating_a_service_account
- create credentials file
 - https://cloud.google.com/storage/docs/authentication#generating-a-private-key
- run script to register service account with Terra..
 - https://github.com/broadinstitute/terra-tools/tree/master/scripts/register_service_account
 - amend relative path to credentials file to absolute path in docker run command provided above. ie..
 - -v /home/mitchac/credentials.json:/svc.json 
- Grant your service account 'writer' permissions on your workspace.
- Upload the credentials.json file for your service account to files area in this workspace.
- Grant your service account 'reader' permissions on the relevant methods in the Broad Methods Repository.

Copy the credentials.json file to the disk of your notebook instance.. 

In [1]:
!gsutil cp gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/notebooks/credentials.json .
!ls

Copying gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/notebooks/credentials.json...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
Config-and-run-workflow.ipynb	  credentials.json
Cost-estimator-gbp-summary.ipynb  runtest.ipynb


Authenticate your service account..

In [1]:
!gcloud auth activate-service-account --key-file credentials.json
!gcloud auth list

Activated service account credentials for: [terra-api@maximal-dynamo-308105.iam.gserviceaccount.com]
                              Credentialed Accounts
ACTIVE  ACCOUNT
        pet-114473713848060875587@firstterrabillingaccount.iam.gserviceaccount.com
*       terra-api@maximal-dynamo-308105.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [2]:
import requests
import os
import json

Run disk size tests..

In [36]:
token = os.popen('gcloud auth --account=terra-api@maximal-dynamo-308105.iam.gserviceaccount.com print-access-token').read().rstrip()
head = {'accept': '*/*',"Content-Type": "application/json", 'Authorization': 'Bearer {}'.format(token)}
myUrl = 'https://api.firecloud.org/api/workspaces/firstterrabillingaccount/singlem-pilot-2/method_configs/singlem/singlem-single-task'

workflow_configs = [{'methodConfigVersion':24,'disk_size_multiplier':2},{'methodConfigVersion':25,'disk_size_multiplier':3}]

for workflow_config in workflow_configs:

    data = {
      "deleted": False,
      "inputs": {
        "SingleM_SRA.GCloud_User_Key_File": "\"gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/sa-private-key.json\"",
        "SingleM_SRA.AWS_User_Key": "",
        "SingleM_SRA.singlem.dockerImage": "",
        "SingleM_SRA.SRA_accession_num": "this.sra_accession",
        "SingleM_SRA.GCloud_Paid": "true",
        "SingleM_SRA.disk_size_floor": "10",
        "SingleM_SRA.disk_size_multiplier": str(workflow_config['disk_size_multiplier']),
        "SingleM_SRA.metagenome_size_in_gbp": "this.metagenome_size_in_gbp",
        "SingleM_SRA.Download_Method_Order": "\"gcp-cp aws-http prefetch\"",
        "SingleM_SRA.AWS_User_Key_Id": ""
      },
      "methodConfigVersion": int(workflow_config['methodConfigVersion']),
      "methodRepoMethod": {
        "methodName": "singlem-single-task",
        "methodVersion": 26,
        "methodNamespace": "singlem",
        "methodUri": "agora://singlem/singlem-single-task/26",
        "sourceRepo": "agora"
      },
      "name": "singlem-single-task",
      "namespace": "singlem",
      "outputs": {
        "SingleM_SRA.SingleM_tables": "this.singlem_table"
      },
      "prerequisites": {},
      "rootEntityType": "autotest_sample"
    }

    response = requests.put(myUrl, data=json.dumps(data), headers=head)

    print(response.json())
    
    data_sets = ["small", "medium", "large"]
    for data_set in data_sets:

      data={
        "methodConfigurationNamespace": "singlem",
        "methodConfigurationName": "singlem-single-task",
        "entityType": "autotest_sample_set",
        "entityName": str(data_set),
        "expression": "this.autotest_samples",
        "useCallCache": False,
        "deleteIntermediateOutputFiles": True,
        "useReferenceDisks": True,
        "workflowFailureMode": "NoNewCalls"
      }
      response = requests.post('https://api.firecloud.org/api/workspaces/firstterrabillingaccount/singlem-pilot-2/submissions', data=json.dumps(data), headers=head)
      print(response.json())

{'extraInputs': [], 'invalidInputs': {}, 'invalidOutputs': {}, 'methodConfiguration': {'deleted': False, 'inputs': {'SingleM_SRA.GCloud_User_Key_File': '"gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/sa-private-key.json"', 'SingleM_SRA.AWS_User_Key': '', 'SingleM_SRA.singlem.dockerImage': '', 'SingleM_SRA.SRA_accession_num': 'this.sra_accession', 'SingleM_SRA.GCloud_Paid': 'true', 'SingleM_SRA.disk_size_floor': '10', 'SingleM_SRA.metagenome_size_in_gbp': 'this.metagenome_size_in_gbp', 'SingleM_SRA.disk_size_multiplier': '2', 'SingleM_SRA.Download_Method_Order': '"gcp-cp aws-http prefetch"', 'SingleM_SRA.AWS_User_Key_Id': ''}, 'methodConfigVersion': 24, 'methodRepoMethod': {'methodName': 'singlem-single-task', 'methodVersion': 26, 'methodNamespace': 'singlem', 'methodUri': 'agora://singlem/singlem-single-task/26', 'sourceRepo': 'agora'}, 'name': 'singlem-single-task', 'namespace': 'singlem', 'outputs': {'SingleM_SRA.SingleM_tables': 'this.singlem_table'}, 'prerequisites': {}, 'rootEntity

{'header': {'entityType': 'autotest_sample', 'inputExpressions': [{'expression': '2', 'wdlName': 'SingleM_SRA.disk_size_multiplier'}, {'expression': '10', 'wdlName': 'SingleM_SRA.disk_size_floor'}, {'expression': 'this.sra_accession', 'wdlName': 'SingleM_SRA.SRA_accession_num'}, {'expression': 'true', 'wdlName': 'SingleM_SRA.GCloud_Paid'}, {'expression': '"gcp-cp aws-http prefetch"', 'wdlName': 'SingleM_SRA.Download_Method_Order'}, {'expression': '"gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/sa-private-key.json"', 'wdlName': 'SingleM_SRA.GCloud_User_Key_File'}, {'expression': 'this.metagenome_size_in_gbp', 'wdlName': 'SingleM_SRA.metagenome_size_in_gbp'}]}, 'request': {'deleteIntermediateOutputFiles': True, 'entityName': 'medium', 'entityType': 'autotest_sample_set', 'expression': 'this.autotest_samples', 'memoryRetryMultiplier': 1.0, 'methodConfigurationName': 'singlem-single-task', 'methodConfigurationNamespace': 'singlem', 'useCallCache': False, 'useReferenceDisks': True, 'workflow

{'extraInputs': [], 'invalidInputs': {}, 'invalidOutputs': {}, 'methodConfiguration': {'deleted': False, 'inputs': {'SingleM_SRA.GCloud_User_Key_File': '"gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/sa-private-key.json"', 'SingleM_SRA.AWS_User_Key': '', 'SingleM_SRA.singlem.dockerImage': '', 'SingleM_SRA.SRA_accession_num': 'this.sra_accession', 'SingleM_SRA.GCloud_Paid': 'true', 'SingleM_SRA.disk_size_floor': '10', 'SingleM_SRA.metagenome_size_in_gbp': 'this.metagenome_size_in_gbp', 'SingleM_SRA.disk_size_multiplier': '3', 'SingleM_SRA.Download_Method_Order': '"gcp-cp aws-http prefetch"', 'SingleM_SRA.AWS_User_Key_Id': ''}, 'methodConfigVersion': 25, 'methodRepoMethod': {'methodName': 'singlem-single-task', 'methodVersion': 26, 'methodNamespace': 'singlem', 'methodUri': 'agora://singlem/singlem-single-task/26', 'sourceRepo': 'agora'}, 'name': 'singlem-single-task', 'namespace': 'singlem', 'outputs': {'SingleM_SRA.SingleM_tables': 'this.singlem_table'}, 'prerequisites': {}, 'rootEntity

{'header': {'entityType': 'autotest_sample', 'inputExpressions': [{'expression': '10', 'wdlName': 'SingleM_SRA.disk_size_floor'}, {'expression': 'this.sra_accession', 'wdlName': 'SingleM_SRA.SRA_accession_num'}, {'expression': 'true', 'wdlName': 'SingleM_SRA.GCloud_Paid'}, {'expression': '"gcp-cp aws-http prefetch"', 'wdlName': 'SingleM_SRA.Download_Method_Order'}, {'expression': '3', 'wdlName': 'SingleM_SRA.disk_size_multiplier'}, {'expression': '"gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/sa-private-key.json"', 'wdlName': 'SingleM_SRA.GCloud_User_Key_File'}, {'expression': 'this.metagenome_size_in_gbp', 'wdlName': 'SingleM_SRA.metagenome_size_in_gbp'}]}, 'request': {'deleteIntermediateOutputFiles': True, 'entityName': 'medium', 'entityType': 'autotest_sample_set', 'expression': 'this.autotest_samples', 'memoryRetryMultiplier': 1.0, 'methodConfigurationName': 'singlem-single-task', 'methodConfigurationNamespace': 'singlem', 'useCallCache': False, 'useReferenceDisks': True, 'workflow